In [1]:
import os
import sys
os.path.dirname(sys.executable)

'D:\\softwares\\Anaconda'

In [2]:
import sys
!{sys.executable} -m pip install opencv-python

In [3]:
import sys
!{sys.executable} -m pip install Pillow

In [4]:
import sys
!{sys.executable} -m pip install pyzbar

In [73]:
import cv2
import datetime as dt
from pyzbar import pyzbar
from time import sleep
import time
import threading
from datetime import datetime
with open("barcode_result.txt", mode ='r') as file:
    lines = file.read().splitlines()
last_line = lines[-1]
date_time = last_line[12:]
date_time_obj = datetime.strptime(date_time, "%d %B %Y, %A, %H:%M:%S")
date_time_now = dt.datetime.now()
time = date_time_now - date_time_obj
time_seconds = time.total_seconds()
print(time_seconds)
if(time_seconds > 200.0):
    print('true')
else:
    print('false')

221.951014
true


In [55]:
import cv2
import datetime as dt
from pyzbar import pyzbar
from time import sleep
import time
import threading
from datetime import timedelta
from datetime import datetime

def check_validity(barcode_data):
    if(len(barcode_data) == 11):
        if(barcode_data[2] == "K"):
            if(barcode_data[0:3] in ["C2K", "E2K", "I2K", "C1K", "E1K"]):
                return True
            else:
                return False
        else:
            return False
    else:
        return False

def check_last():
    with open("barcode_result.txt", mode ='r') as file:
        lines = file.read().splitlines()
    last_line = lines[-1]
    date_time = last_line[12:]
    date_time_obj = datetime.strptime(date_time, "%d %B %Y, %A, %H:%M:%S")
    date_time_now = dt.datetime.now()
    time = date_time_now - date_time_obj
    file.close()
    return time.total_seconds()

def check_same(to_check):
    with open("barcode_result.txt", mode ='r') as file:
        lines = file.read().splitlines()
    last_line = lines[-1]
    last_check = last_line[:11]
    if(to_check == last_check):
        return True

def read_barcode(frame):
    barcodes = pyzbar.decode(frame)
    for barcode in barcodes:
        font = cv2.FONT_HERSHEY_DUPLEX
        x, y , w, h = barcode.rect
        barcode_info = barcode.data.decode('utf-8')
        last_time = check_last()
        cv2.rectangle(frame, (x, y),(x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, barcode_info, (x + 6, y - 6), font, 2, (255, 255, 255), 1)
        if(check_validity(barcode_info)):
            if((last_time > 5.0) and not (check_same(barcode_info) and (last_time < 100.0))):
                with open("barcode_result.txt", mode ='a') as file:
                    current_datetime = dt.datetime.now()
                    string_date = current_datetime.strftime(" %d %B %Y, %A, %H:%M:%S\n")
                    file.write(barcode_info + string_date)
                file.close()
                print(barcode_info)
            elif(check_same(barcode_info) and (last_time < 100.0)):
                cv2.putText(frame, "ATTENDANCE LOGGED", (x - 100, y + 100), font, 1, (0, 0, 255), 1)
            else:
                cv2.putText(frame, "PLEASE WAIT A FEW SECONDS", (x - 100, y + 100), font, 1, (0, 0, 255), 1)
        else:
            cv2.putText(frame, "WRONG BARCODE", (x - 100, y + 100), font, 1, (0, 0, 255), 1)
    return frame

def main():
    camera = cv2.VideoCapture(1)
    ret, frame = camera.read()
    while ret:
        ret, frame = camera.read()
        frame = read_barcode(frame)
        cv2.imshow('PBL Project', frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break        
    camera.release()
    cv2.destroyAllWindows()
    
if __name__ == '__main__':
    main()

In [70]:
#to see how many times one has logged attendence 

to_read = input("Enter roll number to check: ")
date_check = input("date to check on: ")
count = 0
date_read = datetime.strptime(date_check, "%d %B %Y")
with open("barcode_result.txt", mode ='r') as file:
    lines = file.read().splitlines()
for line in lines:
    roll_number = line[0:11]
    date_time = line[12:24]
    date_time_obj = datetime.strptime(date_time, "%d %B %Y")
    if(roll_number == to_read):
        if(date_read == date_time_obj):
            count += 1
print(count)

Enter roll number to check: E2K20103958
date to check on: 03 July 2021
4
